In [52]:
# while datetime.datetime.now() != five_minutes_later: print(1)
print(datetime.datetime(2022,1,7,15,55) == five)
# five_minutes_later
five = (current + datetime.timedelta(minutes = 5)).replace(microsecond = 0).replace(second = 0)
five

False


datetime.datetime(2022, 1, 7, 16, 0)

In [53]:
current

datetime.datetime(2022, 1, 7, 15, 55, 6, 194388)

In [34]:
from datetime import datetime
import datetime

current = datetime.datetime.now()
ten_minutes_later = current + datetime.timedelta(minutes=10)

print(str(ten_minutes_later))



2022-01-07 16:05:06.194388


In [17]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [18]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}

In [19]:
def download(url, params={}, headers={}, method='GET', limit=3):
    try:
        session = Session()
        resp = session.request(method, url,
                               params=params if method.upper() == 'GET' else '',
                               data=params if method.upper() == 'POST' else '',
                               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(60)  # Server Error이기 때문에 delay를 두고 실행하기 위해서 사용한다.
            # 보통은 5분에 한 번꼴로 random하게 되도록 설정한다.
            download(url, params, headers, method, limit - 1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.reason)
            print(e.response.headers)

    return resp

In [20]:
#(current + datetime.timedelta(minutes = 5)).strftime("%Y.%m.%d %H:%M")

In [21]:
# current = datetime.datetime.now()
# five_min = current + datetime.timedelta(minutes=5)




def naver_crawl(acode, current):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    pages = [1]
    title_result=[]

    seen_date = current.strftime("%Y.%m.%d %H:%M")
#     print(TIME_STOP)
    #seen_date = db.naver_title_crawl_1114.find({"code":acode}).sort('date', -1).limit(1)
    
    #sort 전 어떤 변수에 저장해서, 
    #코드 기준으로 가져와서 데이트 정렬
    #seen_date = list(seen_date)
    #if not seen_date:
        
#     seen_date = five_min
    TIME_STOP = (current - datetime.timedelta(minutes = 5)).strftime("%Y.%m.%d %H:%M")
#     print(seen_date)

#     TF = False

    for page in pages:
            #title_result=[]
        pages.append(page + 1)
        url = 'https://finance.naver.com/item/board.nhn?code={}&page={}'.format(acode,page)
        resp = download(url, headers = headers)
        dom = BeautifulSoup(resp.content, 'lxml')
        table = dom.find('table', {'class' : 'type2'})
        tt = table.select('tbody > tr')
        for k in tt[2:]:
            if len(k.select('td > span')) > 0:
                date = k.select('td > span')[0].text
                print('hi', date)
                if TIME_STOP >= date or date == seen_date : #date <= seen_date or TIME_STOP == date:
                    #씬이랑 긁어온 날짜랑은 씬이 더 커야지. 데이트보다 timestop 작아야함.
                    print('범위 밖 ㅃㅇ')
                    break
#                     TF = True
#                     break

                title = k.select('td.title > a')[0]['title']
                views = k.select('td > span')[1].text
                pos = k.select('td > strong')[0].text
                neg = k.select('td > strong')[1].text
                # print(title)

                put_data = {
                                     'code':acode,
                                     'date':date,
                                    'title':title,
                                      'views':views,
                                      'pos':pos,
                                       'neg':neg
                                                 }
                print(put_data)
                title_result.append(put_data)
                
        #if TF:
            #break
        return title_result
        #else:
#         print(title_result)

#5분단위 리스트로 


In [6]:
result = naver_crawl(ACODE['네이버'], datetime.datetime.now())
# ACODE.keys()

hi 2022.01.08 15:48
범위 밖 ㅃㅇ


In [26]:
def message():
    start = datetime.datetime.now()
    print("스케쥴 실행중...")
    for key,value in ACODE.items():
        result = naver_crawl(value,datetime.datetime.now())
        
    end = datetime.datetime.now()
    print("실행 : ", end - start)

In [27]:
import time
import random
import datetime
from kafka import KafkaProducer
#20분 -> 25분까지 25분 30초에 다시 돌기 시작 -> 
bootstrap_servers = ['localhost:9092']  # kafka broker ip
topicName = 'NAVER'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

job = schedule.every(60).seconds.do(message)
while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    if datetime.datetime.now() == NOW + datetime.timedelta(minutes = 1):

        producer.send(topicName, str(job()).encode())

        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        producer.send(topicName, tim.encode())
        break
#5분단위 크롤링 -> 올리고 새로 쓰고 지우고 (이 친구인가,,,) 
#프로듀서 오늘 9시부터 4시까지 긁고 -> 컨슈머에서 긁어와서 쓰기. (실시간....)
#실행 순간 첫 페이지부터, 실시간으로 되려면 생길 때 마다

스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:26
{'code': '035720', 'date': '2022.01.08 16:26', 'title': '여기가', 'views': '2', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:26
{'code': '035720', 'date': '2022.01.08 16:26', 'title': '카카오 시총봐라 희대의 거품주 닷컴버블 때 보다 더하다 ', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:23
{'code': '035720', 'date': '2022.01.08 16:23', 'title': '7만 카카오 갑니다.', 'views': '2', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 16:27
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 16:27
범위 밖 ㅃㅇ
hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 09:01
범위 밖 ㅃㅇ
hi 2022.01.07 19:13
범위 밖 ㅃㅇ
hi 2022.01.08 13:57
범위 밖 ㅃㅇ
hi 2022.01.08 14:42
범위 밖 ㅃㅇ
hi 2022.01.08 16:05
범위 밖 ㅃㅇ
hi 2022.01.08 12:36
범위 밖 ㅃㅇ
hi 2022.01.08 15:20
범위 밖 ㅃㅇ
hi 2022.01.08 09:55
범위 밖 ㅃㅇ
hi 2022.01.08 16:22
범위 밖 ㅃㅇ
hi

hi 2022.01.08 07:56
범위 밖 ㅃㅇ
hi 2022.01.07 21:44
범위 밖 ㅃㅇ
스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:29
{'code': '035720', 'date': '2022.01.08 16:29', 'title': '잘못 매수해서', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:26
{'code': '035720', 'date': '2022.01.08 16:26', 'title': '여기가', 'views': '5', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:26
{'code': '035720', 'date': '2022.01.08 16:26', 'title': '카카오 시총봐라 희대의 거품주 닷컴버블 때 보다 더하다 ', 'views': '7', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:23
범위 밖 ㅃㅇ
hi 2022.01.08 16:27
{'code': '035420', 'date': '2022.01.08 16:27', 'title': '차분하니 기다리세요', 'views': '5', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:23
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 16:29
{'code': '005930', 'date': '2022.01.08 16:29', 'title': '전과4범 형수뽀찢이 비방은 아니지 진실이다', 'views': '4', 'pos': '0', '

hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 09:01
범위 밖 ㅃㅇ
hi 2022.01.07 19:13
범위 밖 ㅃㅇ
hi 2022.01.08 13:57
범위 밖 ㅃㅇ
hi 2022.01.08 14:42
범위 밖 ㅃㅇ
hi 2022.01.08 16:05
범위 밖 ㅃㅇ
hi 2022.01.08 12:36
범위 밖 ㅃㅇ
hi 2022.01.08 15:20
범위 밖 ㅃㅇ
hi 2022.01.08 09:55
범위 밖 ㅃㅇ
hi 2022.01.08 16:22
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:06
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 04:11
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 16:07
범위 밖 ㅃㅇ
hi 2022.01.08 07:56
범위 밖 ㅃㅇ
hi 2022.01.07 21:44
범위 밖 ㅃㅇ
스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:31
{'code': '035720', 'date': '2022.01.08 16:31', 'title': '진짜 절망~~~~', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:29
{'code': '035720', 'date': '2022.01.08 16:29', 'title': '잘못 매수해서', 'views': '4', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:26
범위 밖 ㅃㅇ
hi 2022.0

hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 09:01
범위 밖 ㅃㅇ
hi 2022.01.07 19:13
범위 밖 ㅃㅇ
hi 2022.01.08 13:57
범위 밖 ㅃㅇ
hi 2022.01.08 14:42
범위 밖 ㅃㅇ
hi 2022.01.08 16:05
범위 밖 ㅃㅇ
hi 2022.01.08 12:36
범위 밖 ㅃㅇ
hi 2022.01.08 15:20
범위 밖 ㅃㅇ
hi 2022.01.08 09:55
범위 밖 ㅃㅇ
hi 2022.01.08 16:22
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:06
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 04:11
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 16:07
범위 밖 ㅃㅇ
hi 2022.01.08 07:56
범위 밖 ㅃㅇ
hi 2022.01.07 21:44
범위 밖 ㅃㅇ
스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:33
{'code': '035720', 'date': '2022.01.08 16:33', 'title': '이딴 50배 해묵은거   버려라', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:31
{'code': '035720', 'date': '2022.01.08 16:31', 'title': '진짜 절망~~~~', 'views': '5', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:29
범위 밖 ㅃㅇ

hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 16:36
{'code': '005930', 'date': '2022.01.08 16:36', 'title': '조선일보', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:36
{'code': '005930', 'date': '2022.01.08 16:36', 'title': '이건 공식적인 경고다 경고. 또 몸에 불지르고 날뛰어 보시든가. ', 'views': '6', 'pos': '0', 'neg': '1'}
hi 2022.01.08 16:34
{'code': '005930', 'date': '2022.01.08 16:34', 'title': '이번주에 안철수였다면, 다음주는 윤석열테마', 'views': '5', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:34
{'code': '005930', 'date': '2022.01.08 16:34', 'title': '삼성도 호재 뜨면...?', 'views': '7', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:34
{'code': '005930', 'date': '2022.01.08 16:34', 'title': '지옥철타볼라고 타냐고', 'views': '13', 'pos': '1', 'neg': '2'}
hi 2022.01.08 16:33
{'code': '005930', 'date': '2022.01.08 16:33', 'title': '민주당 지지자들', 'views': '6', 'pos': '2', 'neg': '0'}
hi 2022.01.08 16:31
범위 밖 ㅃㅇ
hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 09:01
범위 밖 ㅃㅇ
hi 2022.01.07 19:13
범위 밖 ㅃㅇ
hi 2022.01.08 13:57
범위 밖 ㅃㅇ
hi 2022.01.08 14:42
범위 밖 ㅃㅇ


hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 04:11
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 16:07
범위 밖 ㅃㅇ
hi 2022.01.08 07:56
범위 밖 ㅃㅇ
hi 2022.01.07 21:44
범위 밖 ㅃㅇ
스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:36
{'code': '035720', 'date': '2022.01.08 16:36', 'title': '이번주에 안철수였다면, 다음주는 윤석열테마', 'views': '5', 'pos': '1', 'neg': '0'}
hi 2022.01.08 16:33
범위 밖 ㅃㅇ
hi 2022.01.08 16:37
{'code': '035420', 'date': '2022.01.08 16:37', 'title': 'Oci는 66만원갔다가 5만원이되고 지금 10만원이 되었습니다', 'views': '6', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:35
{'code': '035420', 'date': '2022.01.08 16:35', 'title': '네이버', 'views': '4', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:34
{'code': '035420', 'date': '2022.01.08 16:34', 'title': '네이버의 현적정가 20만원', 'views': '6', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:27
범위 밖 ㅃㅇ
hi 2022.01.08 16:34
{'code': '036570', 'date': '202

hi 2022.01.08 16:21
범위 밖 ㅃㅇ
hi 2022.01.08 09:01
범위 밖 ㅃㅇ
hi 2022.01.07 19:13
범위 밖 ㅃㅇ
hi 2022.01.08 13:57
범위 밖 ㅃㅇ
hi 2022.01.08 14:42
범위 밖 ㅃㅇ
hi 2022.01.08 16:36
{'code': '004170', 'date': '2022.01.08 16:36', 'title': '니깅리ㅡ', 'views': '1', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:05
범위 밖 ㅃㅇ
hi 2022.01.08 12:36
범위 밖 ㅃㅇ
hi 2022.01.08 15:20
범위 밖 ㅃㅇ
hi 2022.01.08 09:55
범위 밖 ㅃㅇ
hi 2022.01.08 16:22
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:06
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 04:11
범위 밖 ㅃㅇ
hi 2022.01.08 15:59
범위 밖 ㅃㅇ
hi 2022.01.08 16:07
범위 밖 ㅃㅇ
hi 2022.01.08 07:56
범위 밖 ㅃㅇ
hi 2022.01.07 21:44
범위 밖 ㅃㅇ
스케쥴 실행중...
hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:39
{'code': '035720', 'date': '2022.01.08 16:39', 'title': '현정권과 똑같은 먹튀기업 1.2위 카카오 sk', 'views': '3', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:36
범위 밖

hi 2022.01.08 12:45
범위 밖 ㅃㅇ
hi 2022.01.08 14:58
범위 밖 ㅃㅇ
hi 2022.01.08 15:52
범위 밖 ㅃㅇ
hi 2022.01.08 14:24
범위 밖 ㅃㅇ
hi 2022.01.08 16:16
범위 밖 ㅃㅇ
hi 2022.01.08 14:54
범위 밖 ㅃㅇ
hi 2022.01.08 16:42
{'code': '035720', 'date': '2022.01.08 16:42', 'title': '카톡으로 어디까지 우려먹을줄알았냐?', 'views': '6', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:42
{'code': '035720', 'date': '2022.01.08 16:42', 'title': '15만원에 물린 개미들 앞으로 대응 방안', 'views': '10', 'pos': '0', 'neg': '0'}
hi 2022.01.08 16:39
{'code': '035720', 'date': '2022.01.08 16:39', 'title': '현정권과 똑같은 먹튀기업 1.2위 카카오 sk', 'views': '7', 'pos': '0', 'neg': '1'}
hi 2022.01.08 16:36
범위 밖 ㅃㅇ
hi 2022.01.08 16:37
범위 밖 ㅃㅇ
hi 2022.01.08 16:34
범위 밖 ㅃㅇ
hi 2022.01.08 13:32
범위 밖 ㅃㅇ
hi 2022.01.08 16:42
{'code': '005930', 'date': '2022.01.08 16:42', 'title': '후보수준이', 'views': '5', 'pos': '1', 'neg': '1'}
hi 2022.01.08 16:39
{'code': '005930', 'date': '2022.01.08 16:39', 'title': 'acdc부적절  발언   1건 접수  완료 ', 'views': '19', 'pos': '0', 'neg': '1'}
hi 2022.01.08 16:39
{'code': '005

KeyboardInterrupt: 

In [ ]:
import schedule
import time

# step2.실행할 함수 선언
def message():
    print("스케쥴 실행중...")
    for key,value in ACODE.items():
        result = naver_crawl(value,datetime.datetime.now())
        #for i in result:
            #print(i)
#         return

    
# step3.실행 주기 설정
job = schedule.every(60).seconds.do(message)




   


# step4.스캐쥴 시작
count = 0
#current = datetime.datetime.now()
#five_minutes_later = current + datetime.timedelta(minutes=5)




#3시 5분 - 3시 10분 => 15분 예측
# 현재 3시 5분이면, 3시부터- 3시 4분 => 5분이나 10분의 데이터 예측..?
while True:
    
    NOW = datetime.datetime.now()
    schedule.run_pending()
    #time.sleep(1)
    
    

   # if datetime.datetime.now().strftime("%H:%M") == NOW.strftime("%H:%M"):
   #     schedule.cancel_job(job)